In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

/home/janek/.local/share/virtualenvs/rosetta-WKmHhL03/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
import keras.layers as L
from keras.models import Model
import numpy as np
import pandas as pd

RANDOM_STATE=42
np.random.seed(RANDOM_STATE)
tf.set_random_seed(RANDOM_STATE)

In [3]:
START = '^'
END = '$'

SIZE = 100_000
LATENT_DIM = 512
EMBEDDING_DIM = 16
EPOCHS = 20
BATCH_SIZE = 128

In [4]:
def create_equations_df(size, min_value=0, max_value=9999, operations={'+': np.add, '-': np.subtract}):
    df = pd.DataFrame()
    df['a'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['b'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['op'] = np.random.choice(list(operations.keys()), size)
    df['result'] = np.zeros(size, dtype='int')
    for symbol, calc in operations.items():
        df.loc[df.op == symbol, 'result'] = calc(df[df.op == symbol]['a'], df[df.op == symbol]['b'])
        
    df['input_texts'] = df.a.astype(str) + df.op + df.b.astype(str)
    df['target_texts'] = START + df.result.astype(str) + END
    return df

In [5]:
df = create_equations_df(SIZE) #, min_value=0, max_value=50, operations={'+': np.add})

In [6]:
corpus = pd.concat([df.input_texts, df.target_texts])

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=None, filters=None, char_level=True)
tokenizer.fit_on_texts(corpus)
df['input_sequences'] = tokenizer.texts_to_sequences(df.input_texts)
df['target_sequences'] = tokenizer.texts_to_sequences(df.target_texts)

In [8]:
X = keras.preprocessing.sequence.pad_sequences(df.input_sequences, padding='post')
y = keras.preprocessing.sequence.pad_sequences(df.target_sequences, padding='post')
y_t_output = keras.utils.to_categorical(y[:,1:], num_classes=len(tokenizer.word_index)+1)
x_t_input = y[:,:-1]

max_len_input = X.shape[1]
max_len_target = x_t_input.shape[1]
nr_tokens = y_t_output.shape[2]

In [9]:
tokenizer.word_index
nr_tokens
y_t_output.shape
len(tokenizer.word_index)

{'1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '^': 10,
 '$': 11,
 '0': 12,
 '-': 13,
 '+': 14}

15

(100000, 6, 15)

14

In [10]:
encoder_gru = L.GRU(LATENT_DIM, return_state=True, name='encoder_gru')
decoder_gru = L.GRU(LATENT_DIM, return_sequences=True, return_state=True, name='decoder_gru')
decoder_dense = L.Dense(nr_tokens, activation='softmax', name='decoder_outputs')

shared_embedding = L.Embedding(nr_tokens, EMBEDDING_DIM, mask_zero=True, name='shared_embedding')

encoder_inputs = L.Input(shape=(max_len_input, ), dtype='int32', name='encoder_inputs')
encoder_embeddings = shared_embedding(encoder_inputs)
_, encoder_states = encoder_gru(encoder_embeddings)

decoder_inputs = L.Input(shape=(max_len_target, ), dtype='int32', name='decoder_inputs')
decoder_mask = L.Masking(mask_value=0)(decoder_inputs)
decoder_embeddings_inputs = shared_embedding(decoder_mask)
decoder_embeddings_outputs, _ = decoder_gru(decoder_embeddings_inputs, initial_state=encoder_states) 
decoder_outputs = decoder_dense(decoder_embeddings_outputs)


model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

inference_encoder_model = Model(encoder_inputs, encoder_states)
    
inference_decoder_state_inputs = L.Input(shape=(LATENT_DIM, ), dtype='float32', name='inference_decoder_state_inputs')
inference_decoder_embeddings_outputs, inference_decoder_states = decoder_gru(
    decoder_embeddings_inputs,
    initial_state=inference_decoder_state_inputs
)
inference_decoder_outputs = decoder_dense(inference_decoder_embeddings_outputs)

inference_decoder_model = Model(
    [decoder_inputs, inference_decoder_state_inputs], 
    [inference_decoder_outputs, inference_decoder_states]
)

In [11]:
model.summary()
inference_decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     (None, 6)            0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 6)            0           decoder_inputs[0][0]             
__________________________________________________________________________________________________
encoder_inputs (InputLayer)     (None, 9)            0                                            
__________________________________________________________________________________________________
shared_embedding (Embedding)    multiple             240         encoder_inputs[0][0]             
                                                                 masking_1[0][0]                  
__________

In [12]:
model.compile(optimizer=keras.optimizers.Adam(clipnorm=1.), loss='categorical_crossentropy')

In [13]:
model.fit([X, x_t_input], y_t_output, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE)

Train on 90000 samples, validate on 10000 samples
Epoch 1/20
90000/90000 [==============================] - 108s 1ms/step - loss: 1.6467 - val_loss: 1.5114
Epoch 2/20
90000/90000 [==============================] - 107s 1ms/step - loss: 1.3663 - val_loss: 1.2778
Epoch 3/20
90000/90000 [==============================] - 107s 1ms/step - loss: 1.2369 - val_loss: 1.2174
Epoch 4/20
90000/90000 [==============================] - 107s 1ms/step - loss: 1.1637 - val_loss: 1.1554
Epoch 5/20
90000/90000 [==============================] - 108s 1ms/step - loss: 1.1049 - val_loss: 1.0689
Epoch 6/20
90000/90000 [==============================] - 112s 1ms/step - loss: 1.0709 - val_loss: 1.0429
Epoch 7/20
90000/90000 [==============================] - 111s 1ms/step - loss: 1.0435 - val_loss: 1.0458
Epoch 8/20
90000/90000 [==============================] - 107s 1ms/step - loss: 1.0289 - val_loss: 1.1021
Epoch 9/20
90000/90000 [==============================] - 107s 1ms/step - loss: 1.0114 - val_loss: 1.0

In [14]:
def decode_sequence(input_seq):
    states_value = inference_encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, max_len_target))
    target_seq[0, 0] = tokenizer.word_index[START]
    
    tokens = {idx: token for (token, idx) in tokenizer.word_index.items()}
    
    decoded_sequence = ''
    for i in range(max_len_target):
        output_tokens, output_states = inference_decoder_model.predict(
            [target_seq, states_value]
        )
        
        sampled_token_idx = np.argmax(output_tokens[0, 0, :])
        sampled_token = tokens.get(sampled_token_idx, '.')
        if sampled_token == END:
            break
        decoded_sequence += sampled_token
            
        target_seq[0, 0] = sampled_token_idx
        states_value = output_states
    
    return decoded_sequence 

In [15]:
for calc in ['1+1', '9+11', '21+34', '359+468', '1359+468', '1-1', '19-1', '34-359', '1359-468']:
    input_seq = keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([calc]), 
        padding='post', 
        maxlen=X.shape[1]
    )
    print(f"{calc}=got: {decode_sequence(input_seq)}, exp: {eval(calc)}")

1+1=got: 179, exp: 2
9+11=got: -7089, exp: 20
21+34=got: 190, exp: 55
359+468=got: 836, exp: 827
1359+468=got: 1800, exp: 1827
1-1=got: 1985, exp: 0
19-1=got: 178, exp: 18
34-359=got: -305, exp: -325
1359-468=got: 799, exp: 891
